In [10]:
# Importing modules
import numpy as np
import pandas as pd
import altair as alt
import seaborn as sns

import matplotlib.pyplot as plt
#plt.style.use('ggplot')
%matplotlib inline  
#plt.rcParams['figure.figsize'] = (10, 6) 

# Reading in data
# fire_data = pd.read_csv('fire_data_cleaned.csv')
air_quality_data = pd.read_csv('air_quality_data_cleaned.csv')

In [11]:
def date_col(dataframe):
    date = []
    month_date = []
    year_date = []
    for index in range(len(dataframe)):
        date_index = []
        month_index = []
        year = str(dataframe.year[index])
        month = str(dataframe.month[index])
        day = str(dataframe.day[index])
        if len(month) == 1:
            month_index = [year, '0' + month]
            if len(day) == 1:
                date_index = [year, '0' + month, '0' + day]
            else:
                date_index = [year, '0' + month, day]
        else:
            month_index = [year, month]
            if len(day) == 1:
                date_index = [year, month, '0' + day]
            else:
                date_index = [year, month, day]
        date += ['-'.join(date_index)]
        month_date += ['-'.join(month_index)]
        year_date.append(year)
    dataframe['date'] = date
    dataframe['month date'] = month_date
    dataframe['year date'] = year_date
    return dataframe

# fire_data = date_col(fire_data)
air_quality_data = date_col(air_quality_data)

In [12]:
# Grouping by yearly median and average to create baseline
air_quality_year = air_quality_data.groupby('year date')
air_quality_year_med = air_quality_year.median()
air_quality_year_avg = air_quality_year.mean()

air_quality_month = air_quality_data.groupby('month date')
air_quality_month_med = air_quality_month.median()
air_quality_month_avg = air_quality_month.mean()

# Creating baseline AQI column
year_med_baseline_dict = air_quality_year_med['DAILY_AQI_VALUE'].to_dict()
air_quality_data['AQI yr med baseline'] = air_quality_data['year date'].map(year_med_baseline_dict)

# Creating comparison AQI column
air_quality_data['AQI yr med compare'] = air_quality_data['DAILY_AQI_VALUE'] - air_quality_data['AQI yr med baseline']

air_quality_data

,year,month,day,date,state,SITE_LATITUDE,SITE_LONGITUDE,Daily Mean PM2.5 Concentration,DAILY_AQI_VALUE,PERCENT_COMPLETE,month date,year date,AQI yr med baseline,AQI yr med compare
0,2001,1,13,2001-01-13,AZ,34.058475,-109.441259,0.8,3,100.0,2001-01,2001,30.0,-27.0
1,2001,1,16,2001-01-16,AZ,34.058475,-109.441259,1.7,7,100.0,2001-01,2001,30.0,-23.0
2,2001,2,21,2001-02-21,AZ,34.058475,-109.441259,1.3,5,100.0,2001-02,2001,30.0,-25.0
3,2001,2,24,2001-02-24,AZ,34.058475,-109.441259,1.6,7,100.0,2001-02,2001,30.0,-23.0
4,2001,2,27,2001-02-27,AZ,34.058475,-109.441259,2.1,9,100.0,2001-02,2001,30.0,-21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2411560,2021,12,18,2021-12-18,WY,44.565217,-110.400996,0.3,1,100.0,2021-12,2021,25.0,-24.0
2411561,2021,12,21,2021-12-21,WY,44.565217,-110.400996,0.6,3,100.0,2021-12,2021,25.0,-22.0
2411562,2021,12,24,2021-12-24,WY,44.565217,-110.400996,0.1,0,100.0,2021-12,2021,25.0,-25.0
2411563,2021,12,27,2021-12-27,WY,44.565217,-110.400996,0.3,1,100.0,2021-12,2021,25.0,-24.0


In [17]:
air_quality_data_2019_mask = air_quality_data['year'] >=2019
air_quality_data_2019 = air_quality_data.loc[air_quality_data_2019_mask]

air_quality_day_2019 = air_quality_data_2019.groupby('date', as_index = False)
air_quality_day_avg_2019 = air_quality_day_2019.mean()
air_quality_day_avg_2019

,date,year,month,day,SITE_LATITUDE,SITE_LONGITUDE,Daily Mean PM2.5 Concentration,DAILY_AQI_VALUE,PERCENT_COMPLETE,AQI yr med baseline,AQI yr med compare
0,2019-01-01,2019.0,1.0,1.0,40.455095,-116.772492,10.026954,36.544474,100.0,22.0,14.544474
1,2019-01-02,2019.0,1.0,2.0,40.563161,-116.760544,10.567313,39.481994,100.0,22.0,17.481994
2,2019-01-03,2019.0,1.0,3.0,40.105080,-116.487243,11.656890,41.230315,100.0,22.0,19.230315
3,2019-01-04,2019.0,1.0,4.0,40.606745,-116.794184,13.216578,45.385027,100.0,22.0,23.385027
4,2019-01-05,2019.0,1.0,5.0,40.626679,-116.808347,11.913600,43.061333,100.0,22.0,21.061333
...,...,...,...,...,...,...,...,...,...,...,...
1536,2023-03-17,2023.0,3.0,17.0,39.592632,-105.738107,3.530769,14.846154,100.0,22.0,-7.153846
1537,2023-03-18,2023.0,3.0,18.0,39.592632,-105.738107,3.500000,14.615385,100.0,22.0,-7.384615
1538,2023-03-19,2023.0,3.0,19.0,39.592632,-105.738107,3.453846,14.461538,100.0,22.0,-7.538462
1539,2023-03-20,2023.0,3.0,20.0,39.592632,-105.738107,4.553846,19.000000,100.0,22.0,-3.000000


In [20]:
# Disabling data limit
alt.data_transformers.disable_max_rows()

# Creating dropdown menus for month and year
months = ['1','2','3','4','5','6','7','8','9','10','11','12']
years = ['2019', '2020', '2021']

month_dropdown = alt.binding_select(options = list(range(1,13)) + [None], labels = months + ['All'], name = 'Month')
month_selection = alt.selection_point(fields = ['month'], bind = month_dropdown)
year_dropdown = alt.binding_select(options = list(range(2019,2022)) + [None], labels = years + ['All'], name='Year')
year_selection = alt.selection_point(fields = ['year'], bind=year_dropdown)

# Graphs for AQI over time
AQI_day_line = alt.Chart(air_quality_day_avg_2019, width = 900, height = 600).mark_line(clip = True).encode(
    alt.X('date:T').title('Day'),
    alt.Y('DAILY_AQI_VALUE:Q').scale(domain = (10, 88)).title('AQI Value')
).add_params(month_selection, year_selection).transform_filter(year_selection & month_selection)

AQI_yr_med_line = alt.Chart(air_quality_day_avg_2019).mark_line(color = 'red').encode(
    alt.X('date:T').title('Day'),
    alt.Y('AQI yr med baseline:Q').scale(domain = (10, 88))
).add_params(month_selection, year_selection).transform_filter(year_selection & month_selection)

alt.layer(AQI_day_line, AQI_yr_med_line)

alt.LayerChart(...)